In [1]:
import os
import shutil
import xml.etree.ElementTree as ET

In [2]:
import torch

print("Number of GPU: ", torch.cuda.device_count())
print("GPU Name: ", torch.cuda.get_device_name())


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Number of GPU:  1
GPU Name:  NVIDIA GeForce RTX 3050 Laptop GPU
Using device: cuda


In [3]:
word_xml_path = "Dataset/words.xml"
root = ET.parse(word_xml_path).getroot()

for img in root: 
    for tags in img.findall("taggedRectangles"):
        for tag in tags:
            bbox = [
                tag.attrib["x"],
                tag.attrib["y"],
                tag.attrib["width"],
                tag.attrib["height"]
            ]
            print(tag[0].text, bbox)
        break
    break


self ['174.0', '392.0', '274.0', '195.0']
adhesive ['512.0', '391.0', '679.0', '183.0']
address ['184.0', '612.0', '622.0', '174.0']
labels ['863.0', '599.0', '446.0', '187.0']
36 ['72.0', '6.0', '95.0', '87.0']
89m ['247.0', '2.0', '197.0', '88.0']
cls ['792.0', '0.0', '115.0', '81.0']
250 ['200.0', '848.0', '228.0', '139.0']
on ['473.0', '878.0', '165.0', '109.0']
a ['684.0', '878.0', '71.0', '106.0']
roll ['806.0', '844.0', '218.0', '141.0']


In [4]:
img_paths = []
img_sizes = []
img_labels = []
bounding_boxes = []

for img in root:
    bbox_word = []
    label_word = []
    for words in img.findall("taggedRectangles"):
        for word in words:
            if not word[0].text.isalnum():
                continue
            if "é" in word[0].text.lower() or "ñ" in word[0].text.lower():
                continue
            bbox_word.append([
                float(word.attrib["x"]),
                float(word.attrib["y"]),
                float(word.attrib["width"]),
                float(word.attrib["height"])
            ])
            label_word.append(word[0].text.lower())
    
    img_paths.append(img[0].text)
    img_sizes.append((int(img[1].attrib["x"]), int(img[1].attrib["y"])))
    img_labels.append(label_word)
    bounding_boxes.append(bbox_word)
    
print(img_paths[0], img_sizes[0], img_labels[0], bounding_boxes[0])

apanar_06.08.2002/IMG_1261.JPG (1600, 1200) ['self', 'adhesive', 'address', 'labels', '36', '89m', 'cls', '250', 'on', 'a', 'roll'] [[174.0, 392.0, 274.0, 195.0], [512.0, 391.0, 679.0, 183.0], [184.0, 612.0, 622.0, 174.0], [863.0, 599.0, 446.0, 187.0], [72.0, 6.0, 95.0, 87.0], [247.0, 2.0, 197.0, 88.0], [792.0, 0.0, 115.0, 81.0], [200.0, 848.0, 228.0, 139.0], [473.0, 878.0, 165.0, 109.0], [684.0, 878.0, 71.0, 106.0], [806.0, 844.0, 218.0, 141.0]]


In [5]:
def convert_to_yolov8_format(img_paths, img_sizes, bounding_boxes):
    yolov8_data = []
    for img_path, img_size, bounding_box in zip(img_paths, img_sizes, bounding_boxes):
        img_width, img_height = img_size
        labels = []
        
        for bbox in bounding_box:
            x, y, w, h = bbox
            
            center_x = (x + w / 2) / img_width
            center_y = (y + h / 2) / img_height
            nomal_w = w / img_width
            nomal_h = h / img_height
            
            class_id = 0
            
            yolov8_label = f"{class_id} {center_x} {center_y} {nomal_w} {nomal_h}"
            labels.append(yolov8_label)
        
        yolov8_data.append((img_path, labels))
    
    return yolov8_data

yolov8_data = convert_to_yolov8_format(img_paths, img_sizes, bounding_boxes)
yolov8_data[0]

('apanar_06.08.2002/IMG_1261.JPG',
 ['0 0.194375 0.40791666666666665 0.17125 0.1625',
  '0 0.5321875 0.40208333333333335 0.424375 0.1525',
  '0 0.309375 0.5825 0.38875 0.145',
  '0 0.67875 0.5770833333333333 0.27875 0.15583333333333332',
  '0 0.0746875 0.04125 0.059375 0.0725',
  '0 0.2159375 0.03833333333333333 0.123125 0.07333333333333333',
  '0 0.5309375 0.03375 0.071875 0.0675',
  '0 0.19625 0.7645833333333333 0.1425 0.11583333333333333',
  '0 0.3471875 0.7770833333333333 0.103125 0.09083333333333334',
  '0 0.4496875 0.7758333333333334 0.044375 0.08833333333333333',
  '0 0.571875 0.7620833333333333 0.13625 0.1175'])

In [6]:
from sklearn.model_selection import train_test_split
seed = 1
vaid_rate = 0.2
test_rate = 0.125
train_data, remain_data = train_test_split(yolov8_data, test_size=vaid_rate, random_state=seed ,shuffle=True)
valid_data, test_data = train_test_split(remain_data, test_size=test_rate,random_state= seed ,shuffle=True)

In [7]:
len(train_data), len(valid_data), len(test_data)

(200, 43, 7)

In [8]:
def save_data(data, scr_img_dir, save_dir):
    os.makedirs(save_dir, exist_ok=True)
    os.makedirs(os.path.join(save_dir, "images"), exist_ok=True)
    os.makedirs(os.path.join(save_dir, "labels"), exist_ok=True)
    
    for img_path, labels in data:
        shutil.copy(
            os.path.join(scr_img_dir, img_path),
            os.path.join(save_dir, "images")
        )
        
        img_name = os.path.basename(img_path)
        img_name = os.path.splitext(img_name)[0]
        
        with open(os.path.join(save_dir, "labels", f"{img_name}.txt"), "w") as f:
            for label in labels:
                f.write(f"{label}\n")

In [9]:
os.makedirs("Dataset/yolo_data", exist_ok=True)
save_train_dir = os.path.join("Dataset/yolo_data", "train")
save_test_dir = os.path.join("Dataset/yolo_data", "test")
save_valid_dir = os.path.join("Dataset/yolo_data", "val")

save_data(train_data, "Dataset", save_train_dir)
save_data(valid_data, "Dataset", save_valid_dir)
save_data(test_data, "Dataset", save_test_dir)

In [1]:
import os
from ultralytics import YOLO
import torch

yaml_path = os.path.join("Dataset/yolo_data", "data.yaml")

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = YOLO("yolov8s.yaml").load("yolov8s.pt").to(device)

img_size = 512
batch = 2

torch.cuda.empty_cache()

results = model.train(
    data=yaml_path,
    epochs=100,
    imgsz=img_size,
    batch= batch,
    device=device
)


Transferred 355/355 items from pretrained weights
Ultralytics YOLOv8.2.82  Python-3.12.5 torch-2.4.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
engine\trainer: task=detect, mode=train, model=yolov8s.yaml, data=Dataset/yolo_data\data.yaml, epochs=100, time=None, patience=100, batch=2, imgsz=512, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=None, name=train, exist_ok=False, pretrained=yolov8s.pt, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_tx

100%|██████████| 6.25M/6.25M [00:00<00:00, 13.6MB/s]


AMP: checks passed 


c:\Users\hieuh\AppData\Local\Programs\Python\Python312\Lib\site-packages\ultralytics\engine\trainer.py:268: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(enabled=self.amp)
train: Scanning C:\Users\hieuh\Downloads\Project\Scene-Text-Recognition\Dataset\yolo_data\train\labels.cache... 200 images, 9 backgrounds, 0 corrupt: 100%|██████████| 200/200 [00:00<?, ?it/s]
val: Scanning C:\Users\hieuh\Downloads\Project\Scene-Text-Recognition\Dataset\yolo_data\val\labels.cache... 43 images, 0 backgrounds, 0 corrupt: 100%|██████████| 43/43 [00:00<?, ?it/s]


Plotting labels to runs\detect\train\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 512 train, 512 val
Using 8 dataloader workers
Logging results to runs\detect\train
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all         43        186        0.9      0.826       0.88      0.678



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     90/100      0.57G     0.7272     0.5411     0.9707          5        512: 100%|██████████| 100/100 [00:06<00:00, 16.39it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:00<00:00, 25.94it/s]

                   all         43        186      0.901      0.828      0.886      0.678


Closing dataloader mosaic


In [2]:
from ultralytics import YOLO

model_path = "runs\\detect\\train42\\weights\\best.pt"
model = YOLO(model=model_path)

model.val()

Ultralytics YOLOv8.2.82  Python-3.12.5 torch-2.4.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
YOLOv8s summary (fused): 168 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs


val: Scanning C:\Users\hieuh\Downloads\Project\Scene-Text-Recognition\Dataset\yolo_data\val\labels.cache... 43 images, 0 backgrounds, 0 corrupt: 100%|██████████| 43/43 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:05<00:00,  1.84s/it]


                   all         43        186       0.94      0.841       0.89      0.685
Speed: 1.7ms preprocess, 11.2ms inference, 0.0ms loss, 7.6ms postprocess per image
Results saved to runs\detect\val3


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001EE38DD7B60>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

In [3]:
from ultralytics import YOLO

model_path = "runs\\detect\\train42\\weights\\best.pt"
model = YOLO(model=model_path)

img_path = "Dataset\\yolo_data\\val\\images\\from_joanne.jpg"

test = model.predict(img_path, show=True)
print(test)


image 1/1 C:\Users\hieuh\Downloads\Project\Scene-Text-Recognition\Dataset\yolo_data\val\images\from_joanne.jpg: 224x512 26 texts, 64.6ms
Speed: 0.0ms preprocess, 64.6ms inference, 7.5ms postprocess per image at shape (1, 3, 224, 512)
[ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'text'}
obb: None
orig_img: array([[[109, 164, 179],
        [ 87, 139, 155],
        [ 96, 141, 155],
        ...,
        [104, 136, 147],
        [108, 139, 138],
        [110, 141, 134]],

       [[ 88, 137, 153],
        [ 65, 110, 124],
        [ 69, 108, 123],
        ...,
        [ 75, 102, 112],
        [ 82, 106, 104],
        [ 84, 109,  99]],

       [[101, 135, 151],
        [ 71, 104, 119],
        [ 73,  99, 115],
        ...,
        [ 85,  99, 105],
        [ 89, 103,  97],
        [ 91, 106,  92]],

       ...,

       [[147, 174, 184],
        [139, 165, 172],
        [140, 162, 168],
       

In [ ]:
def split_bbox(img_path, img_labels, bboxes, save_dir,):
    os.makedirs(save_dir, exist_ok=True)
    
    count = 0
    labels = []
    for img_path, img_label, bbox  in zip(img_path, img_labels, bboxes):
        